## Travel Recommendation Agent using BeeAI Framework

## Basic Prompt Template for Travel Recommendations

In [1]:
from pydantic import BaseModel, Field

from beeai_framework.template import PromptTemplate, PromptTemplateInput


# Define the input schema for our travel template
class TravelRecommendationInput(BaseModel):
    destination: str
    interests: str
    duration: str


# Create a basic travel recommendation template
travel_template: PromptTemplate = PromptTemplate(
    PromptTemplateInput(
        schema=TravelRecommendationInput,
        template="""
Destination: {{destination}}
Traveler's Interests: {{interests}}
Trip Duration: {{duration}}

Provide tailored travel recommendations for the destination above, focusing on the traveler's 
specific interests and considering the trip duration. Include suggestions for attractions,
activities, and local experiences.""",
    )
)

In [2]:
# Render the template with sample data
prompt = travel_template.render(
    TravelRecommendationInput(
        destination="Kyoto, Japan",
        interests="Traditional culture, gardens, and food",
        duration="5 days",
    )
)

# Print the rendered prompt
print(prompt)


Destination: Kyoto, Japan
Traveler's Interests: Traditional culture, gardens, and food
Trip Duration: 5 days

Provide tailored travel recommendations for the destination above, focusing on the traveler's 
specific interests and considering the trip duration. Include suggestions for attractions,
activities, and local experiences.


## advanced Template with Conditional Logic

In [3]:
class TravelConstraint(BaseModel):
    type: str  # "budget", "accessibility", "season", etc.
    details: str


class AdvancedTravelInput(BaseModel):
    destination: str
    interests: str
    duration: str
    constraints: list[TravelConstraint]
    first_time_visitor: bool


# Create an advanced travel template with conditional sections
advanced_travel_template: PromptTemplate = PromptTemplate(
    PromptTemplateInput(
        schema=AdvancedTravelInput,
        template="""
Destination: {{destination}}
Traveler's Interests: {{interests}}
Trip Duration: {{duration}}

{{#first_time_visitor}}
Note: This is the traveler's first visit to this destination.
{{/first_time_visitor}}

{{#constraints.0}}
Important constraints to consider:
{{#constraints}}
- {{type}}: {{details}}
{{/constraints}}
{{/constraints.0}}

Provide tailored travel recommendations for the destination above, focusing on the traveler's 
specific interests and considering the trip duration. Include suggestions for attractions,
activities, and local experiences.
{{#first_time_visitor}}
Also include must-see attractions for first-time visitors.
{{/first_time_visitor}}""",
    )
)

In [4]:
# Render the template with sample data
advanced_prompt = advanced_travel_template.render(
    AdvancedTravelInput(
        destination="Barcelona, Spain",
        interests="Architecture, beaches, and cuisine",
        duration="7 days",
        constraints=[
            TravelConstraint(type="Budget", details="Mid-range, approximately €150/day"),
            TravelConstraint(type="Season", details="Summer (July-August)"),
        ],
        first_time_visitor=True,
    )
)

# Print the rendered prompt
print(advanced_prompt)


Destination: Barcelona, Spain
Traveler's Interests: Architecture, beaches, and cuisine
Trip Duration: 7 days

Note: This is the traveler's first visit to this destination.

Important constraints to consider:
- Budget: Mid-range, approximately €150/day
- Season: Summer (July-August)

Provide tailored travel recommendations for the destination above, focusing on the traveler's 
specific interests and considering the trip duration. Include suggestions for attractions,
activities, and local experiences.
Also include must-see attractions for first-time visitors.



## gearing up ChatModel for Travel Recommendations

In [5]:
from beeai_framework.backend.chat import ChatModel
from beeai_framework.backend.message import UserMessage, AssistantMessage, SystemMessage
from beeai_framework.backend.types import ChatModelOutput

# Set up credentials for WatsonX
WATSONX_API_KEY = ""
WATSONX_PROJECT_ID = ""
WATSONX_API_URL = "https://us-south.ml.cloud.ibm.com"


# Create a ChatModel instance
model = ChatModel.from_name(
    "watsonx:ibm/granite-3-8b-instruct",
    options={
        "project_id": WATSONX_PROJECT_ID,
        "api_key": WATSONX_API_KEY,
        "api_base": WATSONX_API_URL,
    },
)

In [6]:
# Send a simple travel query to the model
user_message = UserMessage(content="What are the top 3 things to do in Tokyo for a first-time visitor?")

# Get the model's response
output: ChatModelOutput = await model.create(messages=[user_message])

print("Model Response:")
print(output.get_text_content())

Model Response:
1. Visit the Tsukiji Fish Market: Although the inner wholesale market has moved to Toyosu, the outer market remains a bustling hub of food stalls and shops. Here, you can sample some of the freshest sushi and seafood in Tokyo. Don't miss the chance to try tuna belly (otoro) sushi, a local delicacy.

2. Explore Asakusa and Senso-ji Temple: Asakusa is one of Tokyo's oldest districts, offering a glimpse into the city's traditional past. The highlight is Senso-ji, Tokyo's oldest and most significant Buddhist temple. After visiting the temple, take a leisurely stroll along Nakamise Shopping Street, lined with traditional snacks and souvenirs.

3. Experience Shibuya Crossing and Hachiko Statue: Shibuya Crossing is one of the busiest pedestrian crossings in the world, and it's an iconic symbol of Tokyo. After crossing the street, head to Shibuya Station to see the famous Hachiko statue, a loyal Akita dog who waited for his owner for years. The surrounding area is a vibrant sho

## Building conversation memory

In [7]:
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory

# Initialize memory
travel_memory = UnconstrainedMemory()

# Start a conversation about travel plans
await travel_memory.add_many(
    [
        UserMessage(content="I'm planning a trip to Thailand next month."),
    ]
)

# Get the model's response
first_response: ChatModelOutput = await model.create(messages=travel_memory.messages)
print("First Response:")
print(first_response.get_text_content())

# Add the assistant's response to memory and ask a follow-up question
await travel_memory.add_many(
    [
        AssistantMessage(content=first_response.get_text_content()),
        UserMessage(content="What should I pack for the weather there?"),
    ]
)

# Get the model's follow-up response
follow_up_response: ChatModelOutput = await model.create(messages=travel_memory.messages)
print("\nFollow-up Response:")
print(follow_up_response.get_text_content())

First Response:
That sounds like an exciting adventure! Thailand is a beautiful country with a rich culture and diverse landscapes. Here are some tips to help you plan your trip:

1. **Visa Requirements**: Check if you need a visa to enter Thailand. As of my knowledge cutoff date, citizens of many countries can get a 30-day tourist visa exemption upon arrival. However, it's best to verify this information with the Thai embassy or consulate in your country.

2. **Vaccinations**: Ensure you're up-to-date with routine vaccinations. Hepatitis A and Typhoid are recommended for travel to Thailand.

3. **Health Precautions**: Bring a basic travel health kit including pain relievers, antidiarrheal medication, and any personal prescriptions. Also, consider getting travel insurance that covers medical expenses.

4. **Money**: The local currency is the Thai Baht. Credit cards are widely accepted, but it's a good idea to carry some cash for smaller establishments or markets. ATMs are readily avail

## structured travel recommendations

In [8]:
from typing import List, Literal
from pydantic import BaseModel, Field


class Activity(BaseModel):
    name: str = Field(description="Name of the activity or attraction")
    category: Literal["Cultural", "Nature", "Food", "Shopping", "Adventure"] = Field(
        description="Category of the activity"
    )
    estimated_time: str = Field(description="Estimated time needed for this activity")
    description: str = Field(description="Brief description of the activity")


class DayItinerary(BaseModel):
    day_number: int = Field(description="Day number in the itinerary")
    morning: Activity = Field(description="Morning activity")
    afternoon: Activity = Field(description="Afternoon activity")
    evening: Activity = Field(description="Evening activity or dinner recommendation")


class TravelItinerary(BaseModel):
    destination: str = Field(description="The travel destination")
    ideal_for: List[str] = Field(description="Types of travelers this itinerary is ideal for")
    days: List[DayItinerary] = Field(description="Day-by-day itinerary breakdown")
    estimated_budget: str = Field(description="Estimated budget range for this itinerary")
    travel_tips: List[str] = Field(description="Helpful tips specific to this destination")

In [9]:
# Create a user message requesting an itinerary
itinerary_request = UserMessage(
    content="Please create a 3-day itinerary for Bali, Indonesia for a couple interested in both relaxation and adventure."
)

# Generate a structured travel itinerary
itinerary_response = await model.create_structure(schema=TravelItinerary, messages=[itinerary_request])

print("Structured Itinerary:")
print(itinerary_response.object)

Structured Itinerary:
{'days': [{'afternoon': {'category': 'Adventure', 'description': 'Start your day with a visit to the Tegalalang Rice Terrace for a scenic walk and photo opportunities. Afterward, head to the nearby White Water Rafting spot in the Ayung River for an adrenaline-pumping experience. Post-rafting, relax at the nearby natural hot springs, Sacred Monkey Forest Sanctuary in Ubud for a serene evening.', 'estimated_time': '4 hours', 'name': 'Day 1: Ubud'}, 'day_number': 1, 'evening': {'category': 'Cultural', 'description': 'Enjoy a traditional Balinese dance performance at Ubud Palace followed by dinner at a local warung (small restaurant) to savor authentic Indonesian cuisine.', 'estimated_time': '3 hours', 'name': 'Day 1: Ubud'}, 'morning': {'category': 'Adventure', 'description': 'Embark on a bike tour through the lush countryside, visiting local temples and rice paddies. Stop by the Tegenungan Waterfall for a refreshing swim before heading back to your accommodation for

In [21]:
itinerary_response.object

{'days': [{'afternoon': {'category': 'Adventure',
    'description': 'Start your day with a visit to the Tegalalang Rice Terrace for a scenic walk and photo opportunities. Afterward, head to the nearby White Water Rafting spot in the Ayung River for an adrenaline-pumping experience. Post-rafting, relax at the nearby natural hot springs, Sacred Monkey Forest Sanctuary in Ubud for a serene evening.',
    'estimated_time': '4 hours',
    'name': 'Day 1: Ubud'},
   'day_number': 1,
   'evening': {'category': 'Cultural',
    'description': 'Enjoy a traditional Balinese dance performance at Ubud Palace followed by dinner at a local warung (small restaurant) to savor authentic Indonesian cuisine.',
    'estimated_time': '3 hours',
    'name': 'Day 1: Ubud'},
   'morning': {'category': 'Adventure',
    'description': 'Embark on a bike tour through the lush countryside, visiting local temples and rice paddies. Stop by the Tegenungan Waterfall for a refreshing swim before heading back to your ac

## personalizing the travel agent with system prompts

In [10]:
# Define different travel agent personalities
luxury_agent = SystemMessage(
    content="You are a luxury travel advisor specializing in high-end experiences. Your recommendations focus on exclusive, premium options and VIP experiences. Your tone is sophisticated and refined."
)

backpacker_agent = SystemMessage(
    content="You are a budget-conscious backpacker travel guide. You prioritize authentic local experiences, value for money, and off-the-beaten-path recommendations. Your tone is casual and adventurous."
)

family_agent = SystemMessage(
    content="You are a family travel specialist. You focus on child-friendly activities, safety, convenience, and experiences that appeal to multiple generations. Your tone is warm, helpful, and reassuring."
)

In [11]:
# Test the luxury travel agent
luxury_query = UserMessage(content="Recommend a weekend in Paris.")
luxury_output: ChatModelOutput = await model.create(messages=[luxury_agent, luxury_query])

print("Luxury Travel Agent Response:")
print(luxury_output.get_text_content())

# Test the backpacker travel agent
backpacker_query = UserMessage(content="Recommend a weekend in Paris.")
backpacker_output: ChatModelOutput = await model.create(messages=[backpacker_agent, backpacker_query])

print("\nBackpacker Travel Agent Response:")
print(backpacker_output.get_text_content())

Luxury Travel Agent Response:
Certainly, I would be delighted to curate an exquisite weekend itinerary for you in the City of Light.

**Day 1: Arrival & Exclusive City Tour**

Upon your arrival at Le Bourget Airport, a private chauffeur will greet you with a luxury vehicle, whisking you away to your five-star accommodation, the Hôtel Plaza Athénée. This iconic hotel, located on the prestigious Avenue Montaigne, offers unparalleled service and opulent rooms.

In the afternoon, embark on a private guided tour of Paris's most renowned landmarks. Your expert guide will lead you through the Louvre, bypassing the lines with your VIP access, ensuring a serene exploration of the museum's treasures. Post lunch at the hotel's Michelin-starred restaurant, Le Meurice Alain Ducasse, visit the Champs-Élysées and the Arc de Triomphe.

The evening will be spent at the exclusive Café de Flore, where you can enjoy a glass of champagne while observing the Parisian elite. Dine at a private table, prepared

## putting all together: travel planning agent

In [13]:

# Create a function to handle travel planning
async def travel_planning_agent(
    destination: str, 
    interests: str, 
    duration: str, 
    agent_type: str = "standard",
    previous_memory: UnconstrainedMemory = None
):
    # Initialize memory if not provided
    memory = previous_memory if previous_memory else UnconstrainedMemory()
    
    # Select the appropriate system prompt based on agent type
    if agent_type == "luxury":
        system = luxury_agent
    elif agent_type == "backpacker":
        system = backpacker_agent
    elif agent_type == "family":
        system = family_agent
    else:
        system = SystemMessage(content="You are a helpful travel planning assistant.")
    
    # If this is a new conversation, add the system message
    if not previous_memory:
        await memory.add(system)
    
    # Create a travel recommendation using our template
    travel_input = TravelRecommendationInput(
        destination=destination,
        interests=interests,
        duration=duration
    )
    
    prompt = travel_template.render(travel_input)
    
    # Add the templated question to memory
    await memory.add(UserMessage(content=prompt))
    
    # Get a response from the model
    response: ChatModelOutput = await model.create(messages=memory.messages)
    
    # Add the response to memory
    await memory.add(AssistantMessage(content=response.get_text_content()))
    
    return {
        "response": response.get_text_content(),
        "memory": memory
    }

In [20]:
# Example usage of the travel planning agent
result = await travel_planning_agent(
    destination="New Zealand",
    interests="Hiking, nature photography, and local cuisine",
    duration="10 days",
    agent_type="backpacker"
)

print("Travel Planning Agent Response:")
print(result["response"])

# Continue the conversation with a follow-up question
memory = result["memory"]
await memory.add(UserMessage(content="What camera equipment would you recommend for this trip?"))

follow_up: ChatModelOutput = await model.create(messages=memory.messages)
print("\nFollow-up Response:")
print(follow_up.get_text_content())

Travel Planning Agent Response:
Hey there, fellow adventurer! New Zealand is a backpacker's paradise, and I've got a 10-day itinerary that'll have you hiking through stunning landscapes, snapping epic nature photos, and savoring the local flavors. Let's dive right in!

**Day 1-2: Auckland**

Start your Kiwi adventure in Auckland, the "City of Sails." Explore the vibrant Viaduct Harbour and the historic waterfront. Visit the Auckland War Memorial Museum to learn about Maori culture and the country's history. For foodies, Auckland's diverse culinary scene offers a range of options. Try a traditional hangi feast or a seafood platter at one of the many waterfront eateries.

**Day 3-4: Waitomo Glowworm Caves & Rotorua**

Head south to Waitomo, famous for its mesmerizing glowworm caves. Book a Black Water Rafting tour for an unforgettable experience. Then, make your way to Rotorua, the geothermal capital of New Zealand. Visit Wai-O-Tapu to see colorful geysers and mud pools, and explore Te P